In [ ]:
#importando as bibliotecas
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

In [69]:
dados = pd.read_csv('C://fontes_dados//ml//tudo.csv')

In [70]:
##Apenas para fazer testes mais rápidos
dados = dados.query('ano >= 2019')
dados = dados.query('carteira_ativa <= 5000')
dados

,Unnamed: 0,data_base,uf,cliente,ocupacao,modalidade,numero_de_operacoes,carteira_ativa,ano
7866811,3,2019-01-31,AC,1,1,1,15,192,2019
7866813,5,2019-01-31,AC,1,1,5,15,1579,2019
7866814,6,2019-01-31,AC,1,1,1,15,833,2019
7866815,7,2019-01-31,AC,1,1,5,15,1061,2019
7866817,9,2019-01-31,AC,1,1,1,15,2004,2019
...,...,...,...,...,...,...,...,...,...
13989169,2175459,2021-01-30,TO,2,9,9,15,1463,2021
13989171,2175461,2021-01-30,TO,2,9,7,15,3000,2021
13989178,2175468,2021-01-30,TO,2,9,7,15,1136,2021
13989198,2175488,2021-01-30,TO,2,9,7,15,10,2021


In [71]:
dados.to_csv('C://fontes_dados//ml//manipular.csv')

In [38]:
#Mudar o tipo data
dados['data_base'] = pd.to_datetime(dados['data_base'], format='%Y-%m-%d')

dados.dropna(inplace = True)
dados.isnull().sum()

del dados['Unnamed: 0']
del dados['uf']

#Criando uma coluna com a média móvel anual dos valores
dados['media_movel_anual'] = (dados['carteira_ativa'].rolling(365).mean())/100

limite = dados[dados['carteira_ativa'] >= 100000].index
dados.drop(limite, inplace=True)

dados

,data_base,cliente,ocupacao,modalidade,numero_de_operacoes,carteira_ativa,ano,media_movel_anual
11813714,2021-01-31,1,1,1,15,4786,2021,NaN
11813719,2021-01-31,1,1,1,15,8037,2021,NaN
11813720,2021-01-31,1,1,5,15,8680,2021,NaN
11813721,2021-01-31,1,1,2,15,1398,2021,NaN
11813722,2021-01-31,1,1,3,15,5132,2021,NaN
...,...,...,...,...,...,...,...,...
13989178,2021-01-30,2,9,7,15,1136,2021,34.908712
13989193,2021-01-30,2,9,10,15,8347,2021,35.113644
13989195,2021-01-30,2,9,7,15,6623,2021,35.270438
13989198,2021-01-30,2,9,7,15,10,2021,35.041151


In [39]:
dados.dropna(inplace = True)
dados.isnull().sum()

data_base              0
cliente                0
ocupacao               0
modalidade             0
numero_de_operacoes    0
carteira_ativa         0
ano                    0
media_movel_anual      0
dtype: int64

In [40]:
dados.reset_index(drop=True)

,data_base,cliente,ocupacao,modalidade,numero_de_operacoes,carteira_ativa,ano,media_movel_anual
0,2021-01-31,2,9,7,15,2108,2021,35.755068
1,2021-01-31,2,9,8,15,172,2021,35.628658
2,2021-01-31,2,9,9,15,9491,2021,35.668493
3,2021-01-31,2,9,10,15,4514,2021,35.554356
4,2021-01-31,2,9,7,15,883,2021,35.540247
...,...,...,...,...,...,...,...,...
442101,2021-01-30,2,9,7,15,1136,2021,34.908712
442102,2021-01-30,2,9,10,15,8347,2021,35.113644
442103,2021-01-30,2,9,7,15,6623,2021,35.270438
442104,2021-01-30,2,9,7,15,10,2021,35.041151


In [48]:
#Separando quantidade de linhas para treino e teste
qtd_linhas = len(dados)

qtd_linhas_treino = round(.30 * qtd_linhas)
qtd_linhas_teste = qtd_linhas - qtd_linhas_treino  
qtd_linhas_validacao = qtd_linhas -1

info = (
    f"linhas treino= 0:{qtd_linhas_treino}"
    f" linhas teste= {qtd_linhas_treino}:{qtd_linhas_treino + qtd_linhas_teste -1}"
    f" linhas validação= {qtd_linhas_validacao}"
)

info

'linhas treino= 0:132632 linhas teste= 132632:442105 linhas validação= 442105'

In [49]:
#separando as features e labels
features = dados.drop(['carteira_ativa', 'numero_de_operacoes', 'ano', 'data_base'], 1)
labels = dados['carteira_ativa']

In [50]:
#Criando algoritmo para escolher as melhores variáveis (features) para montar o modelo

#Criando uma lista com as variáveis candidatas do dataset
variáveis_candidatas = ('carteira_ativa','media_movel_anual','numero_de_operacoes','modalidade','ocupacao','cliente')

#Estabelecendo a leitura das variáveis candidatas - features
k_best_features = SelectKBest(k='all')

#Estabelecendo pesos para o algoritmo escolher as melhores variáveis
k_best_features.fit_transform(features, labels)
k_best_features_scores = k_best_features.scores_

#Montando os pares de agrupamento estabelecidos
raw_pairs = zip(variáveis_candidatas[1:], k_best_features_scores)
ordered_pairs = list(reversed(sorted(raw_pairs, key=lambda x: x[1])))

#As melhores variáveisjá ordenadas e dentro de um objeto dicionário
k_best_features_final = dict(ordered_pairs[:15])

#Capturando as chaves do dicionário
best_features = k_best_features_final.keys()

#Imprimindo o resultado para avaliação
print ("Melhores Variáveis para o modelo:")
print (k_best_features_final)


Melhores Variáveis para o modelo:
{'modalidade': 2.7115120371334456, 'media_movel_anual': 1.7938877484723288, 'numero_de_operacoes': 1.6726281904046958, 'ocupacao': 1.127015078044089}


In [51]:
melhores_features = dados.loc[:,['modalidade','ocupacao','media_movel_anual','numero_de_operacoes']]
melhores_features

,modalidade,ocupacao,media_movel_anual,numero_de_operacoes
11814947,7,9,35.755068,15
11814948,8,9,35.628658,15
11814956,9,9,35.668493,15
11814957,10,9,35.554356,15
11814962,7,9,35.540247,15
...,...,...,...,...
13989178,7,9,34.908712,15
13989193,10,9,35.113644,15
13989195,7,9,35.270438,15
13989198,7,9,35.041151,15


In [52]:
#Separando os dados que serão usados para treino teste e validação
X_treino = melhores_features[ : qtd_linhas_treino]
X_teste = melhores_features[qtd_linhas_treino : qtd_linhas_treino + qtd_linhas_teste -1]

y_treino = labels[ : qtd_linhas_treino]
y_teste = labels[qtd_linhas_treino : qtd_linhas_treino + qtd_linhas_teste -1]

print( len(X_treino), len(y_treino))

print( len(X_teste), len(y_teste))

132632 132632
309473 309473


In [53]:
# Normalizando os dados de entrada(features)

# Gerando o novo padrão
scaler = MinMaxScaler().fit(melhores_features)
features_scale = scaler.transform(melhores_features)

print("Features: ", features_scale.shape)
print(features_scale)

Features:  (442106, 4)
[[0.46153846 1.         0.99984938 0.        ]
 [0.53846154 1.         0.99984724 0.        ]
 [0.61538462 1.         0.99984791 0.        ]
 ...
 [0.46153846 1.         0.99984115 0.        ]
 [0.46153846 1.         0.99983725 0.        ]
 [0.46153846 1.         0.99983588 0.        ]]


In [54]:
# Normalizando os dados de entrada(features)

# Gerando o novo padrão
scaler = MinMaxScaler()
X_train_scale = scaler.fit_transform(X_treino)  # Normalizando os dados de entrada(treinamento)
X_test_scale  = scaler.transform(X_teste)   

In [55]:
#treinamento usando regressão linear
lr = linear_model.LinearRegression()
lr.fit(X_train_scale, y_treino)
pred = lr.predict(X_test_scale)
cd = r2_score(y_teste, pred)

f'Coeficiente de determinação:{cd * 100:.2f}'

'Coeficiente de determinação:-0.04'

In [56]:
#rede neural
rn = MLPRegressor(max_iter=2000)

rn.fit(X_train_scale, y_treino)
pred= rn.predict(X_test_scale)

cd = rn.score(X_test_scale, y_teste)


f'Coeficiente de determinação:{cd * 100:.2f}'

'Coeficiente de determinação:-0.01'

In [ ]:
#rede neural com ajuste hyper parameters

rn = MLPRegressor()

parameter_space = {
        'hidden_layer_sizes': [(i,) for i in list(range(1, 21))],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam', 'lbfgs'], 
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'adaptive'],
    }

search = GridSearchCV(rn, parameter_space, n_jobs=-1, cv=5)


search.fit(X_train_scale,y_treino)
clf = search.best_estimator_
pred= search.predict(X_test_scale)

cd = search.score(X_test_scale, y_teste)

f'Coeficiente de determinação:{cd * 100:.2f}'

In [57]:
valor_novo = features.tail(1)
valor_novo

,cliente,ocupacao,modalidade,media_movel_anual
13989200,2,9,7,34.960493


In [58]:
#executando a previsão


previsao=scaler.transform(valor_novo)


pred=lr.predict(previsao)

pred

array([1347.63767703])

In [68]:
data_full = dados['data_base']
data = data_full.tail(1)

res_full = dados['carteira_ativa']
res = res_full.tail(1)

dados_previsao = pd.DataFrame({'data_base':data, 'real':res, 'previsao':pred})
dados_previsao.set_index('data_base', inplace=True)

print(dados_previsao)


            real     previsao
data_base                    
2021-01-30    94  1347.637677


In [62]:
dados

,data_base,cliente,ocupacao,modalidade,numero_de_operacoes,carteira_ativa,ano,media_movel_anual
11814947,2021-01-31,2,9,7,15,2108,2021,35.755068
11814948,2021-01-31,2,9,8,15,172,2021,35.628658
11814956,2021-01-31,2,9,9,15,9491,2021,35.668493
11814957,2021-01-31,2,9,10,15,4514,2021,35.554356
11814962,2021-01-31,2,9,7,15,883,2021,35.540247
...,...,...,...,...,...,...,...,...
13989178,2021-01-30,2,9,7,15,1136,2021,34.908712
13989193,2021-01-30,2,9,10,15,8347,2021,35.113644
13989195,2021-01-30,2,9,7,15,6623,2021,35.270438
13989198,2021-01-30,2,9,7,15,10,2021,35.041151
